[예제 2] AdamOptimizer => GradientDescentOptimizer 변경하여 정확도 차이점 확인

1 conv / 1 flatten CNN architecture, 
3 x 3 x 32 filter, padding 있음, 2 x 2 max pooling

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
from datetime import datetime

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print("")
print("train.num = ", mnist.train.num_examples, 
      ", test.num = ", mnist.test.num_examples, 
      ", validation.num = ", mnist.validation.num_examples) 

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [3]:
# shape, type 확인

print("type(mnist) = ", type(mnist), 
      ", type(mnist.train.images) = ", type(mnist.train.images), 
      ", type(mnist.train.labels) = ", type(mnist.train.labels))

print("\ntrain image shape = ", np.shape(mnist.train.images))
print("train label shape = ", np.shape(mnist.train.labels))
print("test image shape = ", np.shape(mnist.test.images))
print("test label shape = ", np.shape(mnist.test.labels))

print("\ntrain image shape = ", mnist.train.images.shape)
print("test image shape = ", mnist.test.images.shape)
print("validation image shape = ", mnist.validation.images.shape)

type(mnist) =  <class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'> , type(mnist.train.images) =  <class 'numpy.ndarray'> , type(mnist.train.labels) =  <class 'numpy.ndarray'>

train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)

train image shape =  (55000, 784)
test image shape =  (10000, 784)
validation image shape =  (5000, 784)


In [4]:
# train data 정규화 및 label 의 one-hot encoding 확인

print('=====================================================================')
print('정규화 확인\n')
print('len(mnist.train.images[0]) = ', len(mnist.train.images[0]))
print(mnist.train.images[0])
print('=====================================================================')
print('One-Hot Encoding 확인\n')
print('len(mnist.train.labels[0]) = ', len(mnist.train.labels[0]))
print(mnist.train.labels[0])  # One-Hot Encoding 확인

정규화 확인

len(mnist.train.images[0]) =  784
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.

In [0]:
# Hyper-Parameter
learning_rate = 0.001  # 학습율
epochs = 30            # 반복횟수
batch_size = 100      # 한번에 입력으로 주어지는 MNIST 개수

In [0]:
# 입력과 출력을 위한 플레이스홀더 정의

X = tf.placeholder(tf.float32, [None, 784])  
T = tf.placeholder(tf.float32, [None, 10])  

# 입력층의 출력 값, 컨볼루션 연산을 위해 reshape 시킴
A1 = X_img = tf.reshape(X, [-1, 28, 28, 1])   # image 28 x 28 x 1 (black/white)

In [0]:
# 1번째 컨볼루션 층, 3X3X32 필터
W2 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))  
#W2 = tf.Variable(tf.random_normal([3, 3, 1, 32]))  
b2 = tf.Variable(tf.constant(0.1, shape=[32]))  
#b2 = tf.Variable(tf.random_normal([32])) 

# 1번째 컨볼루션 연산을 통해 28 X 28 X1  => 28 X 28 X 32 
C2 = tf.nn.conv2d(A1, W2, strides=[1, 1, 1, 1], padding='SAME')

# relu
Z2 = tf.nn.relu(C2+b2)

# 1번째 max pooling을 통해 28 X 28 X 32  => 14 X 14 X 32 
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [0]:
# 14X14 크기를 가진 32개의 activation map을 flatten 시킴
A2_flat = P2_flat = tf.reshape(A2, [-1, 14*14*32])

In [0]:
# 출력층
W3 = tf.Variable(tf.random_normal([14*14*32, 10], stddev=0.01))
b3 = tf.Variable(tf.random_normal([10]))

# 출력층 선형회귀  값 Z3, 즉 softmax 에 들어가는 입력 값
Z3 = logits = tf.matmul(A2_flat, W3) + b3    # 선형회귀 값 Z3

y = A3 = tf.nn.softmax(Z3)

In [0]:
loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T) )

#optimizer = tf.train.AdamOptimizer(learning_rate)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)  

In [0]:
# batch_size X 10 데이터에 대해 argmax를 통해 행단위로 비교함
predicted_val = tf.equal( tf.argmax(A3, 1), tf.argmax(T, 1) )

# batch_size X 10 의 True, False 를 1 또는 0 으로 변환
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

In [12]:
with  tf.Session()  as sess:
    
    sess.run(tf.global_variables_initializer())  # 변수 노드(tf.Variable) 초기화
    
    start_time = datetime.now()
    
    for i in range(epochs):    # 30 번 반복수행
        
        total_batch = int(mnist.train.num_examples / batch_size)  # 55,000 / 100

        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
      
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})    
        
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)             
    
    end_time = datetime.now() 
    
    print("\nelapsed time = ", end_time - start_time) 
    
    # Accuracy 확인
    test_x_data = mnist.test.images    # 10000 X 784
    test_t_data = mnist.test.labels    # 10000 X 10
    
    accuracy_val = sess.run([accuracy], feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy = ", accuracy_val)

epochs =  0 , step =  0 , loss_val =  3.0749927
epochs =  0 , step =  100 , loss_val =  2.32736
epochs =  0 , step =  200 , loss_val =  2.3406591
epochs =  0 , step =  300 , loss_val =  2.3277633
epochs =  0 , step =  400 , loss_val =  2.2810984
epochs =  0 , step =  500 , loss_val =  2.2927265
epochs =  1 , step =  0 , loss_val =  2.2745514
epochs =  1 , step =  100 , loss_val =  2.2755914
epochs =  1 , step =  200 , loss_val =  2.2585044
epochs =  1 , step =  300 , loss_val =  2.2635872
epochs =  1 , step =  400 , loss_val =  2.2408936
epochs =  1 , step =  500 , loss_val =  2.2212248
epochs =  2 , step =  0 , loss_val =  2.1977956
epochs =  2 , step =  100 , loss_val =  2.174067
epochs =  2 , step =  200 , loss_val =  2.1343834
epochs =  2 , step =  300 , loss_val =  2.0899312
epochs =  2 , step =  400 , loss_val =  2.0525248
epochs =  2 , step =  500 , loss_val =  1.9789194
epochs =  3 , step =  0 , loss_val =  1.9756782
epochs =  3 , step =  100 , loss_val =  1.776698
epochs =  3 